In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sp
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
import sunau;
import librosa
import librosa.display
import pandas
import os
np.set_printoptions(suppress=True)

In [2]:
def writeMatrixToFile(self,filename,matrix):
        np.save(filename, matrix);
        os.rename(filename+".npy", filename+".txt");
        print("Written successfully..");
        pass;
    
def loadMatrixFromFile(self,filename):        
    matrix=None;
    if(os.path.isfile(filename)):
        matrix=np.load(filename);       
    return matrix;

def readCSVFile(file):
    data=pandas.read_csv(file,",",header=0, na_values='?', skipinitialspace=True);
    return data;
    pass;

In [3]:
class PreProcessing:
    filename=None;
    y=None;
    sr=None;
    log_enabled=True;
    centroid=None;
    spectro=None;
    spectro_phase=None;
    max_sample_vector_size=660000; 
    duration=30;
    def __init__(self,filename,duration=30):
        #self.log(filename);
        self.filename=filename;
        self.reloadAudioFile(duration);
        self.duration=duration;
        pass;
    
    def reloadAudioFile(self,duration=30):
        self.y, self.sr = librosa.load(self.filename,duration=duration);
        self.y=self.y[:self.max_sample_vector_size];
        pass;
    
    #Short-Term-Fourier trasform
    def getSTFT(self):
        self.stft=librosa.stft(y=self.y);
        return self.stft;
        pass;
    
    #spectro graph
    def getSpectrogram(self):
        stft=self.getSTFT();
        self.spectro, self.spectro_phase = librosa.magphase(stft);        
        return self.spectro, self.spectro_phase;
        pass;
    
    def getCentroid(self):
        self.centroid=librosa.feature.spectral_centroid(y=self.y,sr=self.sr);
        return self.centroid;    

    def getSpectralRolloff(self):
        self.rolloff=librosa.feature.spectral_rolloff(y=self.y, sr=self.sr);
        return self.rolloff;
    
    def getZeroCrossing(self):
        self.zero_crossing_rate=librosa.feature.zero_crossing_rate(self.y);
        return self.zero_crossing_rate;
    
    def getSpectralContrast(self):
        #Jiang, Dan-Ning, Lie Lu, Hong-Jiang Zhang, Jian-Hua Tao, and Lian-Hong Cai. “Music type classification by spectral contrast feature.” In Multimedia and Expo, 2002. ICME‘02. Proceedings. 2002 IEEE International Conference on, vol. 1, pp. 113-116. IEEE, 2002.
        S = np.abs(self.getSTFT());
        self.contrast = librosa.feature.spectral_contrast(S=S, sr=self.sr);
        return self.contrast;
    
    def getMFCC(self):
        self.mfcc = librosa.feature.mfcc(y=self.y, sr=self.sr, hop_length=512, n_mfcc=13);
        return self.mfcc;
    
    def getChroma(self):
        self.chroma = librosa.feature.chroma_stft(y=self.y,sr=self.sr,hop_length=512)
        return self.chroma
    
    def getMelSpec(self):
        self.mel=librosa.feature.melspectrogram(y=self.y, sr=self.sr,n_mels=10);
        return self.mel;
    
    def getRMS(self):
        self.rms=librosa.feature.rms(y=self.y);
        return self.rms;

    def drawRMS(self):
        rms=self.getRMS();
        S,phase=self.getSpectrogram();
        plt.figure()
        plt.subplot(2, 1, 1)
        plt.semilogy(rms.T, label='RMS Energy')
        plt.xticks([])
        plt.xlim([0, rms.shape[-1]])
        plt.legend(loc='best')
        plt.subplot(2, 1, 2)
        librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),y_axis='log', x_axis='time')
        plt.title('log Power spectrogram')
        plt.tight_layout()
        plt.show();
        pass;
    
    def drawSpectrogramWithCentroid(self):
        centroid=self.getCentroid();
        S,phase=self.getSpectrogram();
        plt.figure()
        plt.subplot(2, 1, 1)
        plt.semilogy(centroid.T, label='Spectral centroid')
        plt.ylabel('Hz')
        plt.xticks([])
        plt.xlim([0, centroid.shape[-1]])
        plt.legend()
        plt.subplot(2, 1, 2)
        librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),y_axis='log', x_axis='time')
        plt.title('log Power spectrogram')
        plt.tight_layout();
        plt.show();
        pass;
    
    def drawSpectralRolloff(self):
        rolloff=self.getSpectralRolloff();
        S,phase=self.getSpectrogram();
        plt.figure()
        plt.subplot(2, 1, 1)
        plt.semilogy(rolloff.T, label='Roll-off frequency')
        plt.ylabel('Hz')
        plt.xticks([])
        plt.xlim([0, rolloff.shape[-1]])
        plt.legend()
        plt.subplot(2, 1, 2)
        librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),y_axis='log', x_axis='time')
        plt.title('log Power spectrogram')
        plt.tight_layout();
        plt.show();
        pass;
    
    def drawSpectralContrast(self):
        contrast=self.getSpectralContrast();
        S,phase=self.getSpectrogram();
        S = np.abs(self.getSTFT());
        plt.figure()
        plt.subplot(2, 1, 1)
        librosa.display.specshow(librosa.amplitude_to_db(S,ref=np.max),y_axis='log')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Power spectrogram')
        plt.subplot(2, 1, 2)
        librosa.display.specshow(contrast, x_axis='time')
        plt.colorbar()
        plt.ylabel('Frequency bands')
        plt.title('Spectral contrast')
        plt.tight_layout();
        plt.show();
        pass;
    
    def drawMFCC(self):
        mfccs=self.getMFCC();
        plt.figure(figsize=(6, 4))
        librosa.display.specshow(mfccs, x_axis='time')
        #plt.colorbar()
        plt.title('MFCC')
        plt.tight_layout()
        plt.show();
        pass;
    
    
    def log(self,a,b=None):
        if(self.log_enabled):
            if(b!=None):
                print(a,b);
            else:
                print(a);
        pass;  
    
    def estimate_tempo(self,oenv,sample_rate):
        return librosa.beat.tempo(oenv,sr=sample_rate)

    def tempogram_analysis(self,signal, sample_rate, max_tempo = 320.0,hop_length = 512):
        tempogram = librosa.feature.tempogram(y=signal, sr = sample_rate)
        bpms = librosa.core.tempo_frequencies(tempogram.shape[0], hop_length=hop_length, sr=sample_rate)
        means_over_time = np.mean(tempogram, axis = 1)
        #account for max_tempo
        if max_tempo is not None:
            max_idx = np.argmax(bpms < max_tempo)
            means_over_time[:max_idx] = 0    
        top_five_tempos = np.argpartition(means_over_time, -3)[-3:-1]
        top_five_tempo_values = means_over_time[top_five_tempos]
        return bpms[top_five_tempos], top_five_tempo_values
        
    def getRhythm_features(self):
        oenv = librosa.onset.onset_strength(y=self.y, sr=self.sr)
        tempo = self.estimate_tempo(oenv,self.sr)
        top_five_tempos, top_five_tempo_values = self.tempogram_analysis(self.y, self.sr)
        tempos =  np.append(top_five_tempos, top_five_tempo_values)
        return np.append(tempos, tempo)




In [4]:
class ProcessDataset:
    columns = ["id","type","y_index","y","centroid mean","rolloff mean","zero mean","rms mean","rms var","contrast mean","contrast var", "mfcc1 mean","mfcc1 var",
              "mfcc2 mean","mfcc2 var","mfcc3 mean","mfcc3 var","mfcc4 mean","mfcc4 var","mfcc5 mean","mfcc5 var",
               "mel1 mean","mel1 var","mel2 mean","mel2 var","mel3 mean","mel3 var","mel4 mean","mel4 var","mel5 mean", "mel5 var"]+["chroma_mean_"+str(i) for i in range(1,7)]+ ["chroma_var_"+str(i) for i in range(1,7)]+["bpm_second", "bm_first", "tempo_strength_second"]               
    #columns = ["id","type","y_index","y","centroid mean","centroid var","rolloff mean","rolloff var","zero mean",
    #           "zero var","rms mean","rms var","contrast mean","contrast var","mfcc1 mean","mfcc1 var",
    #           "mfcc2 mean","mfcc2 var","mfcc3 mean","mfcc3 var","mfcc4 mean","mfcc4 var","mfcc5 mean","mfcc5 var",
    #           "mfcc6 mean","mfcc6 var","mfcc7 mean","mfcc7 var","mfcc8 mean","mfcc8 var","mfcc9 mean","mfcc9 var",
    #            "mfcc10 mean","mfcc10 var","mfcc11 mean","mfcc11 var","mfcc12 mean","mfcc12 var","mfcc13 mean","mfcc13 var"];
#     +["chroma_mean_"+str(i) for i in range(1,13)]+ ["chroma_var_"+str(i) for i in range(1,13)]+["bpm_second", "bm_first", "tempo_strength_second"]
    genre_out={"blues":[1,0,0,0,0,0,0,0,0,0],"classical":[0,1,0,0,0,0,0,0,0,0],"country":[0,0,1,0,0,0,0,0,0,0],"disco":[0,0,0,1,0,0,0,0,0,0],"hiphop":[0,0,0,0,1,0,0,0,0,0],"jazz":[0,0,0,0,0,1,0,0,0,0],"metal":[0,0,0,0,0,0,1,0,0,0],"pop":[0,0,0,0,0,0,0,1,0,0],"reggae":[0,0,0,0,0,0,0,0,1,0],"rock":[0,0,0,0,0,0,0,0,0,1]};    
    genre_out_index={"blues":0,"classical":1,"country":2,"disco":3,"hiphop":4,"jazz":5,"metal":6,"pop":7,"reggae":8,"rock":9};    
    dataframe=None;
    mfcc_features=5;
    mel_features=5;
    chroma_features=6;
    dir="../genres";
    genre_dir={"blues":"blues","classical":"classical","country":"country","disco":"disco","hiphop":"hiphop","jazz":"jazz","metal":"metal","pop":"pop","reggae":"reggae","rock":"rock"};
    def __init__(self):
        self.dataframe = pandas.DataFrame(columns=self.columns);
        print("Colums",len(self.columns))
        pass;

    def extractTimberalFeatures(self,genre,audio_number,filename):
        features=[];
        pp=PreProcessing(filename);            
        centroid=pp.getCentroid()[0];
        rolloff=pp.getSpectralRolloff()[0];        
        zero=pp.getZeroCrossing()[0];        
        contrast=pp.getSpectralContrast()[0];        
        rms=pp.getRMS()[0];
        mfcc=pp.getMFCC();
        mel=pp.getMelSpec();
        chroma = pp.getChroma();
        rhythm = pp.getRhythm_features();
        
        features.append(audio_number);
        features.append(genre);        
        features.append(self.genre_out_index[genre]);        
        features.append(self.genre_out[genre]);
        features.append(centroid.mean());
#         features.append(centroid.var()); 
        features.append(rolloff.mean());
#         features.append(rolloff.var()); 
        features.append(zero.mean());
#         features.append(zero.var()); 
        features.append(rms.mean());
        features.append(rms.var()); 
        features.append(contrast.mean());
        features.append(contrast.var()); 
        for i in range(self.mfcc_features):
            features.append(mfcc[i].mean());
            features.append(mfcc[i].var());  
        for i in range(self.mel_features):
            features.append(mel[i].mean());
            features.append(mel[i].var());
        for i in range(self.chroma_features):
            features.append(chroma[i].mean());
            features.append(chroma[i].var());
        #features.append(np.mean(chroma,axis=1).reshape(1,12));
        #features.append(np.var(chroma,axis=1).reshape(1,12));
        #print(np.mean(chroma,axis=1).shape);
        #print("Df",self.dataframe.size)
        #print("Features",np.array(features).shape)
        #print(rhythm.shape)
        for i in range(3):
            features.append(rhythm[i]);
        self.dataframe.loc[self.dataframe.size]=features;
    
    def saveDataFrame(self):
        filename="audiofeatures_numpy_matrix";
        arr=self.dataframe.as_matrix(columns=None);
        np.save(filename,arr);       
        self.dataframe.to_csv("audiofeatures.csv",sep=",")
        print("Written successfully..");
        pass;
    
    def extractFeatures(self):
        print("Extracting Features...");
        percent_completed=0;        
        for k in self.genre_dir:    
            genre=k;
            print("-------------------["+genre+"]-----------------------")
            for i in range(100):
                audio_number="%0.5d"%i;
                filename=self.dir+"/"+self.genre_dir[genre]+"/"+self.genre_dir[genre]+"."+audio_number+".wav";              
                self.extractTimberalFeatures(genre,audio_number,filename);  
                percent_completed=i;
                if(percent_completed%10==0):
                    print("Percent completed:",percent_completed);            
        self.saveDataFrame();
        print("Extraction done");
        pass;
        

In [5]:
p1=ProcessDataset();
p1.extractFeatures();
p1.dataframe

Colums 46
Extracting Features...
-------------------[blues]-----------------------
Percent completed: 0
Percent completed: 10
Percent completed: 20
Percent completed: 30
Percent completed: 40
Percent completed: 50
Percent completed: 60
Percent completed: 70
Percent completed: 80
Percent completed: 90
-------------------[classical]-----------------------
Percent completed: 0
Percent completed: 10
Percent completed: 20
Percent completed: 30
Percent completed: 40
Percent completed: 50
Percent completed: 60
Percent completed: 70
Percent completed: 80
Percent completed: 90
-------------------[country]-----------------------
Percent completed: 0
Percent completed: 10
Percent completed: 20
Percent completed: 30
Percent completed: 40
Percent completed: 50
Percent completed: 60
Percent completed: 70
Percent completed: 80
Percent completed: 90
-------------------[disco]-----------------------
Percent completed: 0
Percent completed: 10
Percent completed: 20
Percent completed: 30
Percent completed

/home/yadnyesh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Written successfully..
Extraction done


,id,type,y_index,y,centroid mean,rolloff mean,zero mean,rms mean,rms var,contrast mean,...,chroma_mean_6,chroma_var_1,chroma_var_2,chroma_var_3,chroma_var_4,chroma_var_5,chroma_var_6,bpm_second,bm_first,tempo_strength_second
0,00000,blues,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1784.808293,3808.046818,0.083024,0.130224,0.002833,15.995900,...,0.100146,0.244700,0.044746,0.247899,0.065782,0.335636,0.083699,129.199219,123.046875,0.697285
46,00001,blues,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1529.004413,3545.433548,0.055998,0.095860,0.002375,15.976383,...,0.095222,0.211930,0.052394,0.201273,0.043756,0.331534,0.096766,69.837416,66.256010,0.535636
92,00002,blues,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1552.846406,3041.990598,0.076251,0.175608,0.002753,26.343662,...,0.073941,0.408669,0.073894,0.547632,0.122644,0.322317,0.055691,78.302557,151.999081,0.604331
138,00003,blues,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1070.101734,2185.494924,0.033294,0.141386,0.006324,29.793587,...,0.106601,0.547292,0.092144,0.657226,0.119275,0.511363,0.092011,64.599609,63.024009,0.547889
184,00004,blues,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1836.253062,3582.540766,0.101533,0.091636,0.002304,17.063747,...,0.096787,0.255970,0.075524,0.193062,0.032114,0.531243,0.131189,66.256010,129.199219,0.544492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45770,00095,rock,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",2009.139574,4254.785667,0.089293,0.079451,0.000346,29.623115,...,0.084771,0.406211,0.035509,0.330961,0.093065,0.217626,0.026933,198.768029,215.332031,0.775188
45816,00096,rock,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",2005.279224,4144.849484,0.097594,0.076443,0.000590,29.228484,...,0.055166,0.347186,0.037476,0.562132,0.088365,0.298203,0.030091,78.302557,234.907670,0.745270
45862,00097,rock,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",2076.527225,4029.196153,0.121735,0.081619,0.000322,28.829981,...,0.097083,0.409299,0.044416,0.457064,0.087882,0.341546,0.039950,123.046875,258.398438,0.787496
45908,00098,rock,9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1398.990121,3014.656784,0.048738,0.083942,0.001211,28.549530,...,0.066175,0.226563,0.040740,0.334850,0.086914,0.379064,0.116438,234.907670,215.332031,0.669523


In [6]:
dir="../genres";
genre_dir={"blues":"blues","classical":"classical","country":"country","disco":"disco","hiphop":"hiphop","jazz":"jazz","metal":"metal","pop":"pop","reggae":"reggae","rock":"rock"};

In [7]:
genre="disco"

audio_number="%0.5d"%0;
filename=dir+"/"+genre_dir[genre]+"/"+genre_dir[genre]+"."+audio_number+".wav";  

pp=PreProcessing(filename); 
s,p=pp.getSpectrogram();
len(s)

1025

In [8]:
genre="disco"
audio_number="%0.5d"%0;
filename=dir+"/"+genre_dir[genre]+"/"+genre_dir[genre]+"."+audio_number+".wav";  

pp=PreProcessing(filename); 
a=librosa.feature.melspectrogram(y=pp.y, sr=pp.sr,n_mels=10)
len(a[0])

1290

In [9]:
genre="disco"
pds=ProcessDataset();

audio_number="%0.5d"%98;
filename=dir+"/"+genre_dir[genre]+"/"+genre_dir[genre]+"."+audio_number+".wav";  
pds.extractTimberalFeatures(genre,audio_number,filename);

audio_number="%0.5d"%99;
filename=dir+"/"+genre_dir[genre]+"/"+genre_dir[genre]+"."+audio_number+".wav";  
pds.extractTimberalFeatures(genre,audio_number,filename);

audio_number="%0.5d"%0;
filename=dir+"/"+genre_dir[genre]+"/"+genre_dir[genre]+"."+audio_number+".wav";  
pds.extractTimberalFeatures(genre,audio_number,filename);

#pds.extractFeatures();
pds.dataframe

Colums 46


,id,type,y_index,y,centroid mean,rolloff mean,zero mean,rms mean,rms var,contrast mean,...,chroma_mean_6,chroma_var_1,chroma_var_2,chroma_var_3,chroma_var_4,chroma_var_5,chroma_var_6,bpm_second,bm_first,tempo_strength_second
0,00098,disco,3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",2417.547068,5384.369095,0.112626,0.122804,0.001554,24.266525,...,0.078891,0.428091,0.064880,0.515520,0.116679,0.484060,0.057560,161.499023,61.523438,0.767122
46,00099,disco,3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",2417.547068,5384.369095,0.112626,0.122804,0.001554,24.266525,...,0.078891,0.428091,0.064880,0.515520,0.116679,0.484060,0.057560,161.499023,61.523438,0.767122
92,00000,disco,3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",2295.772581,4884.139433,0.096195,0.142213,0.004145,18.022094,...,0.091744,0.433943,0.064803,0.492812,0.125874,0.481689,0.089429,57.421875,112.347147,0.708882


In [10]:
def partitionDataFrame(df,ratio):
    df = df.sample(frac=1).reset_index(drop=True)#shuffling rows
    df = df.sample(frac=1).reset_index(drop=True)#again shuffling
    size=df["id"].count();
    limit=int(ratio*size);
    train_ds=df.loc[0:limit];    
    test_ds=df.loc[limit:size];
    train_ds.to_csv("train.csv",sep=",");
    test_ds.to_csv("test.csv",sep=",");
    print("Partitioning done");

In [11]:
df=readCSVFile("audiofeatures.csv");
partitionDataFrame(df,0.9);


Partitioning done
